In [1]:
from google.colab import files
uploaded = files.upload()

Saving SmartResumeSort.ipynb to SmartResumeSort.ipynb


In [2]:
import json, os
from google.colab import _message

# Save current notebook
notebook_path = "/content/SmartResumeSort.ipynb"
notebook_json = _message.blocking_request('get_ipynb')['ipynb']
with open(notebook_path, 'w') as f:
    f.write(json.dumps(notebook_json))

In [3]:
!ls /content/

sample_data  SmartResumeSort.ipynb


In [4]:
!git config --global user.email "srivatsavpv1711@gmail.com"
!git config --global user.name "srivatsav29"

In [7]:
!git clone https://github.com/srivatsav29/SmartResumeSort.git

# Copy your notebook into that repo folder
!cp /content/SmartResumeSort.ipynb /content/SmartResumeSort/

# Commit and push
%cd /content/SmartResumeSort/
!git add .
!git commit -m "Added notebook from Colab"
!git push https://github.com/srivatsav29/SmartResumeSort.git

Cloning into 'SmartResumeSort'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 18.02 KiB | 18.02 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/SmartResumeSort
hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> SmartResumeSort
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached SmartResumeSort
hint: 
hint: See "git help submodule" for more information.
[main 6bcb200] Added notebook from Colab
 2 files changed, 2 insertions(+)
 create mode 160000 SmartResumeSort
 create mode 10064

In [8]:
!rm -rf /content/SmartResumeSort

In [10]:
!git clone https://github.com/srivatsav29/SmartResumeSort.git

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'SmartResumeSort': No such file or directory


In [11]:
%cd /content/

/content


In [12]:
!rm -rf /content/SmartResumeSort

In [13]:
!git clone https://github.com/srivatsav29/SmartResumeSort.git

Cloning into 'SmartResumeSort'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 18.02 KiB | 18.02 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
import json
from google.colab import _message

notebook_name = "SmartResumeSort.ipynb"
notebook_content = _message.blocking_request("get_ipynb")["ipynb"]
with open(f"/content/{notebook_name}", "w") as f:
    f.write(json.dumps(notebook_content))